In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:12 of 0:12:12 remaining]
Chain 1:  10% [0:00:58 of 0:01:04 remaining]
Chain 1:  20% [0:00:53 of 0:01:07 remaining]
Chain 1:  30% [0:00:49 of 0:01:10 remaining]
Chain 1:  40% [0:00:42 of 0:01:10 remaining]
Chain 1:  50% [0:00:35 of 0:01:10 remaining]
Chain 1:  60% [0:00:29 of 0:01:11 remaining]
Chain 1:  70% [0:00:22 of 0:01:12 remaining]
Chain 1:  80% [0:00:15 of 0:01:13 remaining]
Chain 1:  90% [0:00:07 of 0:01:13 remaining]
Chain 1: 100% [0:00:00 of 0:01:13 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:47 of 0:00:52 remaining]
Chain 2:  20% [0:00:49 of 0:01:02 remaining]
Chain 2:  30% [0:00:46 of 0:01:06 remaining]
Chain 2:  40% [0:00:41 of 0:01:09 remaining]
Chain 2:  50% [0:00:35 of 0:01:09 remaining]
Chain 2:  60% [0:00:28 of 0:01:10 remaining]
Chain 2:  70% [0:00:21 of 0:01:11 remaining]
Chain 2:  80% [0:00:14 of 0:01:11 remaining]
Chain 2:  90% [0:00:07 of 0:01:12 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[25.0624 115.042 0.883108; 25.868 115.041 0.885934; … ; 26.133 114.527 0.878285; 28.2363 114.527 0.878285]

[26.4428 113.056 0.927548; 27.75 113.674 0.906779; … ; 29.002 111.252 0.960606; 28.9889 116.447 0.847133]

[27.2074 114.797 0.887829; 26.9059 114.798 0.880363; … ; 25.6592 110.132 0.993172; 25.4869 113.431 0.909508]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS  
     s2  26.1536830 1.994647037 0.01213903529 0.0207063168 9000.000
beta[1] 113.8943924 1.866960111 0.01136195741 0.0350911582 2830.575
beta[2]   0.9046571 0.041141556 0.00025037954 0.0007672626 2875.235

Quantiles:
            2.5%        25.0%       50.0%      75.0%       97.5%   
     s2  22.4654209  24.76565499  26.084992  27.4430118  30.2780465
beta[1] 110.1909996 112.63222709 113.909523 115.1330948 117.6109926
beta[2]   0.8232978   0.87725326   0.904442   0.9324733   0.9862088



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*